## Project 5 : Classification

## Instructions

### Description

Practice classification on the Titanic dataset.

### Grading

For grading purposes, we will clear all outputs from all your cells and then run them all from the top.  Please test your notebook in the same fashion before turning it in.

### Submitting Your Solution

To submit your notebook, first clear all the cells (this won't matter too much this time, but for larger data sets in the future, it will make the file smaller).  Then use the File->Download As->Notebook to obtain the notebook file.  Finally, submit the notebook file on Canvas.


In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

plt.style.use('ggplot')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Introduction

On April 15, 1912, the largest passenger liner ever made collided with an iceberg during her maiden voyage. When the Titanic sank it killed 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships. One of the reasons that the shipwreck resulted in such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others.

Intro Videos: 
https://www.youtube.com/watch?v=3lyiZMeTKIo
and
https://www.youtube.com/watch?v=ItjXTieWKyI 

The `titanic_data.csv` file contains data for `887` of the real Titanic passengers. Each row represents one person. The columns describe different attributes about the person including whether they survived (`0=No`), their age, their passenger-class (`1=1st Class, Upper`), gender, and the fare they paid (£s*). For more on the currency: http://www.statisticalconsultants.co.nz/blog/titanic-fare-data.html

We are going to try to see if there are correlations between the feature data provided (find a best subset of features) and passenger survival.

### Problem 1: Load and understand the data (35 points)

#### Your task (some of this is the work you completed for L14 - be sure to copy that work into here as needed)
Conduct some preprocessing steps to explore the following and provide code/answers in the below cells:
1. Load the `titanic_data.csv` file into a pandas dataframe
2. Explore the data provided (e.g., looking at statistics using describe(), value_counts(), histograms, scatter plots of various features, etc.) 
3. What are the names of feature columns that appear to be usable for learning?
4. What is the name of the column that appears to represent our target?
5. Formulate a hypothesis about the relationship between given feature data and the target
6. How did Pclass affect passenngers' chances of survival?
7. What is the age distribution of survivors?

In [ ]:
# Step 1. Load the `titanic_data.csv` file into a pandas dataframe
df = pd.read_csv("./titanic_data.csv")

df.head()

In [ ]:
# Step 2. Explore the data provided (e.g., looking at statistics using describe(), value_counts(), histograms, scatter plots of various features, etc.) 
print(df.describe(),"\n\n\n")
print(df.value_counts())

In [ ]:
# Pie chart of sex
class_1_total = df.loc[df['Pclass'] == 1, "Pclass"].count()

males = len(df.loc[df["Sex"]=="male"])
females = len(df.loc[df["Sex"]=="female"])
plt.pie((males, females), labels=["Males", "Females"], textprops={'fontsize': 14});
plt.title("Passenger Sex", fontsize=16);
plt.show()

males_survived = len(df.loc[(df["Sex"]=="male") & (df["Survived"]==1)])
females_survived = len(df.loc[(df["Sex"]=="female") & (df["Survived"]==1)])
plt.pie((males_survived, females_survived), labels=["Males", "Females"], textprops={'fontsize': 14});
plt.title("Surviving Passenger Sex", fontsize=16);
plt.show()

# Histogram of age
plt.hist(df['Age'])
plt.xlabel('Ages')
plt.ylabel('Number of Passengers')
plt.title('Passenger Age Distribution')
plt.show()

plt.hist(df['Fare'])
plt.xlabel('Fare')
plt.ylabel('Number of Passengers')
plt.title('Passenger Fare Distribution')
plt.show()

---

**Edit this cell to provide answers to the following steps:**

---

Step 3. What are the names of feature columns that appear to be usable for learning? 
Pclass, Sex, Age, Fare, Siblings/Spouses Aboard, Parents/Children Aboard

Step 4. What is the name of the column that appears to represent our target? 
Survived

Step 5. Formulate a hypothesis about the relationship between given feature data and the target
I think that the women and children will be most likely to survive, as well as the higher passenger classes.

In [ ]:
#Step 6. How did Pclass affect passenngers' chances of survival?
#Show your work with a bar plot, dataframe selection, or visual of your choice.

class_1_total = df.loc[df['Pclass'] == 1, "Pclass"].count()
class_2_total = df.loc[df['Pclass'] == 2, "Pclass"].count()
class_3_total = df.loc[df['Pclass'] == 3, "Pclass"].count()

class_1_survivors = df.loc[(df['Pclass'] == 1) & (df['Survived'] == 1), "Survived"].count()
class_2_survivors = df.loc[(df['Pclass'] == 2) & (df['Survived'] == 1), "Survived"].count()
class_3_survivors = df.loc[(df['Pclass'] == 3) & (df['Survived'] == 1), "Survived"].count()

plt.bar(["Class 1", "Class 2", "Class 3"], [class_1_total, class_2_total, class_3_total], color='red')
plt.bar(["Class 1", "Class 2", "Class 3"], [class_1_survivors, class_2_survivors, class_3_survivors], color='blue')
plt.xlabel('Passenger Class')
plt.ylabel('Number of Passengers')
plt.title('Survived Passengers by Class')
plt.legend(['Total Passengers', 'Survived Passengers'])
plt.show()


In [ ]:
#Step 7. What is the age distribution of survivors?
#Show your work with a dataframe operation and/or histogram plot.

survivor_ages = df.loc[(df['Survived'] == 1), "Age"]

plt.hist(survivor_ages);
plt.title('Age Distribution of Survivors')
plt.xlabel('Age')
plt.ylabel('Number of Survivors')
plt.show()

### Problem 2: transform the data (10 points)
The `Sex` column is categorical, meaning its data are separable into groups, but not numerical. To be able to work with this data, we need numbers, so you task is to transform the `Sex` column into numerical data with pandas' `get_dummies` feature and remove the original categorical `Sex` column.

In [ ]:
sex = pd.get_dummies(df["Sex"])
# df = df.drop("Sex", axis=1)
df = df.assign(Sex=sex["male"])
df

### Problem 3: Classification (30 points)
Now that the data is transformed, we want to run various classification experiments on it. The first is `K Nearest Neighbors`, which you will conduct by:

1. Define input and target data by creating lists of dataframe columns (e.g., inputs = ['Pclass', etc.)
2. Split the data into training and testing sets with `train_test_split()`
3. Create a `KNeighborsClassifier` using `5` neighbors at first (you can experiment with this parameter)
4. Train your model by passing the training dataset to `fit()`
5. Calculate predicted target values(y_hat) by passing the testing dataset to `predict()`
6. Print the accuracy of the model with `score()`

** Note: If you get a python warning as you use the Y, trainY, or testY vector in some of the function calls about "DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )", you can look up how to use trainY.values.ravel() or trainY.values.flatten() or another function, etc.

In [ ]:
inputs = ["Pclass", "Age", "Siblings/Spouses Aboard", "Parents/Children Aboard", "Fare", "Sex"]
target = ["Survived"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[inputs], df[target], test_size=0.2)
X_train

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 5
knn = KNeighborsClassifier(n_neighbors=k)

### Problem 4: Cross validation, classification report (15 points)
- Using the concepts from the 17-model_selection slides and the [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function from scikit-learn, estimate the f-score ([`f1-score`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)) (you can use however many folds you wish). 

    `cross_val_score` is a handy utility, but it can be confusing. It doesn't return a model like `KNeighborsClassifier(...)`. Instead, it uses a model and dataset that you provide and runs the whole train-predict-score process for each of `k` folds (refer to the notes on Cross Validation for more information). The function returns a list of scores, one for each of the k folds. To get to a single score, it is possible to take the mean or median of this list of scores. There are also even do more involved statistical techniques. However, it is also correct to just provide the list of scores.

    By default, the `cross_val_score` utility will apply the default scoring metric (accuracy) to every cross validation fold. To get it to apply `f1-score` instead, you will need to create a "scorer" that calculates f1-scores using [`make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html#sklearn.metrics.make_scorer), and then pass this object to the `scoring` parameter of `cross_val_score`. Since this has a few parts to it, let me just give you that scorer object: ```scorerVar = make_scorer(f1_score, pos_label=1)```

- Using the concepts from the end of the 14-classification slides, output a confusion matrix.

- Also, output a classification report [`classification_report`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) from sklearn.metrics showing more of the metrics: precision, recall, f1-score for both of our classes.

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score, classification_report, make_scorer
from sklearn import model_selection
from sklearn.model_selection import cross_val_score

scorerVar = make_scorer(f1_score, pos_label=1)

knn_scores = cross_val_score(knn, X_train, y_train, cv=k, scoring=scorerVar)
knn_scores.mean()

### Problem 5: Support Vector Machines (15 points)
Now, repeat the above experiment using the using a Support Vector classifier [`lr`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.lr.html) with default parameters (RBF kernel) model in scikit-learn, and output:

- The fit accuracy (using the `score` method of the model)
- The f-score (using the [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function)
- The confusion matrix
- The precision, recall, and f-measure for the 1 class (you can just print the results of the [`classification_report`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) function from sklearn.metrics)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

#create a model object
svc = SVC()
#train our model
svc.fit(X_train, y_train)
#evaluate the model
svc_score = svc.score(X_train, y_train)
#setup to get f-score and cv
svc_scores = cross_val_score(svc, X_train, y_train, cv=k, scoring=scorerVar)
#confusion matrix
svc_y_pred = svc.predict(X_test)
svc_conf_matrix = confusion_matrix(y_test, svc_y_pred)
#classification report
svc_class_report = classification_report(y_test, svc_y_pred)

print(svc_class_report)

### Problem 6: Logistic Regression (15 points)

Now, repeat the above experiment using the [`LogisticRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model in scikit-learn, and output:

- The fit accuracy (using the `score` method of the model)
- The f-score (using the [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function)
- The confusion matrix
- The precision, recall, and f-measure for the 1 class (you can just print the results of the [`classification_report`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) function from sklearn.metrics)

In [ ]:
from sklearn.linear_model import LogisticRegression

#create a model object
lr = LogisticRegression()
#train our model
lr.fit(X_train, y_train)
#evaluate the model 
lr_score = lr.score(X_train, y_train)
#setup to get f-score and cv
lr_scores = cross_val_score(lr, X_train, y_train, cv=k, scoring=scorerVar)
#confusion matrix
lr_y_pred = lr.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_y_pred)
#classification report
lr_class_report = classification_report(y_test, lr_y_pred)

print(lr_class_report)

### Problem 7: Comparision and Discussion (5 points)
Edit this cell to provide a brief discussion (3-5 sentances at most):
1. What was the model/algorithm that performed best for you? 
In terms of f1 score and accuracy, Logistic Regression performed the best.
2. What feaures and hyperparameters were used to achieve that performance?
All features were used except for name.
3. What insights did you gain from your experimentation about the predictive power of this dataset and did it match your original hypothesis about the relationship between given feature data and the target?
I learned that there was a significant correlation between data about each passenger and whether that led them to survive, such as the benefit of being in a higher class, and of being female - my friend tells me the women got loaded into the boats first.

### Questionnaire
1) How long did you spend on this assignment?
<br>3 hours and 15 mins<br>
2) What did you like about it? What did you not like about it?
<br>I enjoyed the data visualization and the theory about what would happen to the passengers. I didn't like how convoluded the process for getting predictions was.<br>
3) Did you find any errors or is there anything you would like changed?
<br>I would like a better explanation on how to work through the cross validation.<br>